In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

In [2]:
dfData = pd.read_csv('events_up_to_01062018.csv')
dfTrain = pd.read_csv('labels_training_set.csv')
dfTest = pd.read_csv('trocafone_kaggle_test.csv')
dfSample = pd.read_csv('trocafone_kaggle_submit_sample_all_0.csv')

/home/federico/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Crea un df con todos los usuarios
dfTest['label'] = 0
dfUsers = pd.concat([dfTrain,dfTest])
dfTest = dfTest.drop('label', axis=1)

In [4]:
#Le asigna a cada usuario su cantidad de eventos dandole mas valor si son mas recientes
def pesoTiempo(time):
    mes = int(time[6])
    dia = int(time[8:10])
    if mes < 5:
        return 1
    return 10 + dia * 0.7
    
dfPE = dfData[['person', 'event', 'timestamp']]
dfPE['puntajeTiempo'] = dfPE['timestamp'].apply(pesoTiempo)
dfPE = pd.get_dummies(dfPE, columns=['event'])
columnasEventos = dfPE.columns.values[3:]
for eventoColumna in columnasEventos:
    dfPE[eventoColumna] *= dfPE['puntajeTiempo']
dfPE = dfPE.drop(['timestamp', 'puntajeTiempo'], axis=1).groupby('person').sum()
dfUsers = dfUsers.sort_values('person').merge(dfPE, on='person')

/home/federico/.local/lib/python2.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [5]:
#Le asigna a cada usuario la region en la que mas eventos tiene
regionesMasEventos = dfData['region'].value_counts().head(9).index.tolist()
def filtrarRegiones(region):
    if pd.isnull(region):
        return 'null'
    if region not in regionesMasEventos:
        return 'Other'
    return region
    
dfPE = dfData[['person', 'region']]
dfPE['regionAcotada'] = dfPE['region'].apply(filtrarRegiones)

lDictRegiones = []
temp = dfPE.drop('region', axis=1).groupby('person')
for name, group in temp:
    regiones = {'null' : 1}
    for row in group.itertuples():
        region = row.regionAcotada
        if region == 'null':
            continue
        if region in regiones:
            regiones[region] += 1
        else:
            regiones[region] = 1
        rMax = max(regiones, key=regiones.get)
    lDictRegiones.append({'person' : name, 'region' : rMax})

dfRegiones = pd.DataFrame(lDictRegiones)
dfRegiones = pd.get_dummies(dfRegiones, columns=['region'])
dfUsers = dfUsers.merge(dfRegiones, on='person')

/home/federico/.local/lib/python2.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [6]:
#Casteo dfUsers a int32
personas = dfUsers['person']
dfUsers.drop('person', axis=1, inplace=True)
dfUsers = dfUsers.astype('int32')
dfUsers['person'] = personas

In [7]:
#Agrego columna con ultima visita al sitio
temp = dfData.groupby('person')['timestamp'].max().to_frame()
dfUsers = dfUsers.merge(temp, on='person')
dfUsers['month'] = dfUsers['timestamp'].str[5:7]
dfUsers['day'] = dfUsers['timestamp'].str[8:10]
dfUsers[['day','month']] = dfUsers[['day','month']].apply(pd.to_numeric)
dfUsers['last_visit'] = dfUsers['month']*30 + dfUsers['day']
dfUsers = dfUsers.drop(['timestamp','month','day'], axis=1)
dfUsers.head(10)

,label,event_ad campaign hit,event_brand listing,event_checkout,event_conversion,event_generic listing,event_lead,event_search engine hit,event_searched products,event_staticpage,...,region_Other,region_Parana,region_Pernambuco,region_Rio Grande do Sul,region_Rio de Janeiro,region_Sao Paulo,region_Unknown,region_null,person,last_visit
0,0,0,0,65,0,21,0,0,0,0,...,0,0,0,0,0,0,1,0,0008ed71,167
1,0,373,511,52,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,00091926,181
2,0,1,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,00091a7a,116
3,0,21,575,147,28,347,0,21,0,0,...,0,0,0,0,0,1,0,0,000ba417,176
4,0,30,0,30,0,30,0,30,272,0,...,0,0,0,0,0,0,0,1,000c79fe,179
5,0,413,449,21,0,360,0,117,0,0,...,0,0,0,0,0,0,0,0,000e4d9e,180
6,0,132,238,21,0,171,0,62,132,0,...,0,0,0,0,0,1,0,0,000e619d,168
7,0,0,0,63,21,63,0,0,360,0,...,0,0,0,0,0,1,0,0,001001be,172
8,0,31,0,31,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0010e89a,180
9,0,30,0,30,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0016c4b5,179


In [8]:
#Agrego columna con ultima compra
temp = dfData.loc[dfData['event'].isin(['conversion','lead','checkout'])]\
             .groupby('person')['timestamp'].max().to_frame()
dfUsers = dfUsers.merge(temp, how='left', on='person')
dfUsers = dfUsers.fillna('000000000000000000')
dfUsers['month'] = dfUsers['timestamp'].str[5:7]
dfUsers['day'] = dfUsers['timestamp'].str[8:10]
dfUsers[['day','month']] = dfUsers[['day','month']].apply(pd.to_numeric)
dfUsers['last_conversion'] = dfUsers['month']*30 + dfUsers['day']
dfUsers = dfUsers.drop(['timestamp','month','day'], axis=1)

In [9]:
#Columna con relacion entre el uso del celular y de la computadora
def filtrarDispositivo(deviceType):
    if deviceType == 'Computer':
        return 1
    elif deviceType == 'Smartphone':
        return 0
    else:
        return 2
    
dfPE = dfData[['person', 'device_type']]
dfPE['dispositivo'] = dfPE['device_type'].apply(filtrarDispositivo)
dfPE = dfPE.loc[dfPE['dispositivo'] != 2].drop('device_type', axis=1).groupby('person').mean()
dfUsers = dfUsers.merge(dfPE, on='person', how='left').fillna(0)

/home/federico/.local/lib/python2.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [10]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

df_y = dfTrain['label']
df_X = dfTrain.drop('label', axis=1).merge(dfUsers, on='person').drop(['person','label'], axis=1)

GBC = GradientBoostingClassifier(random_state=23)
scores = cross_val_score(GBC , df_X, df_y, scoring="roc_auc", cv=5)
scores.mean()

0.8750950884670115

In [11]:
#Pruebo con XGBoost
XGBC = xgb.XGBClassifier(learning_rate =0.075, n_estimators=95, max_depth=4, min_child_weight=6, 
                         gamma=0.3, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic',
                         scale_pos_weight=0.8, seed = 15)

scores = cross_val_score(XGBC , df_X, df_y, scoring="roc_auc", cv=5)
scores.mean()

0.8768002496195443

In [12]:
from sklearn.ensemble import VotingClassifier

VC = VotingClassifier(estimators=[('xgb', XGBC), ('gb', GBC)], voting='soft', weights = [1.8,0.9])
scores = cross_val_score(VC , df_X, df_y, scoring="roc_auc", cv=5)
scores.mean()

0.8773571573918385

In [13]:
GBC2 = GradientBoostingClassifier(random_state=23)
GBC2.fit(df_X, df_y)
df_p = dfTest.merge(dfUsers, on='person').drop(['person','label'], axis=1)
temp = GBC2.predict_proba(df_p)
submit = []
for l in temp:
    submit.append(l[1])
dfTest['label'] = submit

In [14]:
dfTest.to_csv('submitConversionLeadCheckOut_votingGradientXGB.csv', index=False)

In [15]:
pd.read_csv('trocafone_kaggle_submit.csv')

,person,label
0,4886f805,0.006272
1,0297fc1e,0.016259
2,2d681dd8,0.009240
3,cccea85e,0.106488
4,4c8a8b93,0.028730
5,29ebb414,0.007458
6,3dc1950f,0.033337
7,8ea4c165,0.029296
8,d8cfe234,0.027095
9,d6bc64df,0.051664
